# Datenwissenschaft in der Cloud: Der "Azure ML SDK"-Ansatz

## Einführung

In diesem Notebook lernen wir, wie man das Azure ML SDK verwendet, um ein Modell zu trainieren, bereitzustellen und zu nutzen, alles über Azure ML.

Voraussetzungen:
1. Sie haben einen Azure ML-Arbeitsbereich erstellt.
2. Sie haben den [Heart Failure-Datensatz](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) in Azure ML geladen.
3. Sie haben dieses Notebook in Azure ML Studio hochgeladen.

Die nächsten Schritte sind:

1. Erstellen Sie ein Experiment in einem bestehenden Arbeitsbereich.
2. Erstellen Sie einen Compute-Cluster.
3. Laden Sie den Datensatz.
4. Konfigurieren Sie AutoML mit AutoMLConfig.
5. Führen Sie das AutoML-Experiment aus.
6. Analysieren Sie die Ergebnisse und erhalten Sie das beste Modell.
7. Registrieren Sie das beste Modell.
8. Stellen Sie das beste Modell bereit.
9. Nutzen Sie den Endpunkt.

## Azure Machine Learning SDK-spezifische Importe


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## Arbeitsbereich initialisieren
Initialisiere ein Arbeitsbereichsobjekt aus der gespeicherten Konfiguration. Stelle sicher, dass die Konfigurationsdatei unter .\config.json vorhanden ist.


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Erstellen Sie ein Azure ML-Experiment

Lassen Sie uns ein Experiment mit dem Namen 'aml-experiment' im Arbeitsbereich erstellen, den wir gerade initialisiert haben.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## Erstellen Sie einen Compute-Cluster
Sie müssen ein [Compute-Ziel](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) für Ihren AutoML-Lauf erstellen.


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## Daten
Stellen Sie sicher, dass Sie den Datensatz in Azure ML hochgeladen haben und dass der Schlüssel denselben Namen wie der Datensatz trägt.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## AutoML-Konfiguration


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## AutoML Lauf


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## Bereitstellung des besten Modells

Führen Sie den folgenden Code aus, um das beste Modell bereitzustellen. Den Status der Bereitstellung können Sie im Azure ML-Portal einsehen. Dieser Schritt kann einige Minuten dauern.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## Verwenden des Endpunkts
Sie können Eingaben zu dem folgenden Eingabe-Beispiel hinzufügen.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
